## Preprocesamiento de los datos

In [ ]:
import os
cwd = os.getcwd()
os.chdir("../code/utils")
from image_transform import find_best_contours, order_points, four_point_transform
os.chdir(cwd)
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import PIL
import cv2
import imutils

from random import shuffle, choice

from tqdm.notebook import tqdm

#### Defining image paths

In [56]:
input_images_path = "../input/data/original"
destination_images_path = "../input/data/preprocessed"
test_data_percentage = 0.1
classes = ['fraud', 'not-fraud']

#### Transforming each image by identifying contours and cutting edges and saving shuffled transformed data to "preprocess" folder

In [ ]:
%%time
progress_bar = tqdm()
for class_ in classes:
    Path(f"{destination_images_path}/train/{class_}").mkdir(parents=True, exist_ok=True)
    Path(f"{destination_images_path}/test/{class_}").mkdir(parents=True, exist_ok=True)
    imgs = os.listdir(input_images_path+f"/{class_}")
    imgs = [img for img in imgs if img!= 'backup']
    shuffle(imgs)
    progress_bar.reset(total=len(imgs))
    new_size = (1024, 500)

    for i, img in enumerate(imgs, start=1):
        train_or_test = 'test' if choice([0,1,2,3,4,5,6,7,8,9]) == 0 else 'train'
        progress_bar.set_description(f"Transforming image '{img}' from '{class_}' folder and saving in 'preprocess/{train_or_test}' folder")
        img_path = input_images_path + f"/{class_}/" + img
        image = cv2.imread(img_path)
        ratio = image.shape[0] / 1000.0
        orig = image.copy()
        image = imutils.resize(image, height = 1000)
        screenCnt, output_image, gray = find_best_contours(image)
        warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
        tranformed_img = PIL.Image.fromarray(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
        if warped.shape[0] > warped.shape[1]:
            tranformed_img = tranformed_img.rotate(-90, PIL.Image.NEAREST, expand = 1)
        tranformed_img = tranformed_img.resize(new_size)
        if train_or_test == 'test':
            tranformed_img.save(destination_images_path+f"/test/{class_}/{i}.png")
        else:
            tranformed_img.save(destination_images_path+f"/train/{class_}/{i}.png")
        progress_bar.update()

0it [00:00, ?it/s]

CPU times: total: 1min 18s
Wall time: 2min 33s
